## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model files from here: 

* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

With wget:

```bash
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"
wget ${DATA_URL}
wget ${MODEL_URL}
```


In [1]:
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

# IPython automatically substitutes the Python variable $DATA_URL
!wget $DATA_URL
!wget $MODEL_URL

--2025-12-09 14:14:48--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-09T08%3A03%3A20Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-09T07%3A02%3A36Z&ske=2025-12-09T08%3A03%3A20Z&sks=b&skv=2018-11-09&sig=P1BQd9xE%2B48Bdy8zOF7pPcZijTCDSNQwhlL1zv%2BQ8nA%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTI2NjI4OSwibmJmIjoxNzY1MjY0NDg

--2025-12-09 14:15:00--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/c6b83ad5-a901-40e9-bf2c-41ad174c870c?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-09T08%3A10%3A03Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-09T07%3A09%3A43Z&ske=2025-12-09T08%3A10%3A03Z&sks=b&skv=2018-11-09&sig=qtufuQETddRqFp%2FP9YdYivkf%2Bjf%2BztNVmwN0uPnvGSc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTI2NDgwMCwibmJmIjoxNzY1MjY0NTAwLCJwYXR

## Question 1

To be able to use this model, we need to know the name of the input and output nodes. 

What's the name of the output:

* `output`
* `sigmoid`
* `softmax`
* `prediction`

**Answer: `output`**

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

In [1]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [2]:
import onnxruntime

# Define the model path
MODEL_PATH = 'hair_classifier_v1.onnx'

# 1. Create an inference session
session = onnxruntime.InferenceSession(MODEL_PATH)

# 2. Check Input Node Name and Shape
print("### Input Node Information ###")

# The session.get_inputs() method returns a list of NodeArg objects.
# We usually care about the first one (index 0) for single-input models.

input_node = session.get_inputs()[0]
print(f"Name: {input_node.name}")
print(f"Shape: {input_node.shape}")
print(f"Data Type: {input_node.type}")

print("\n-------------------------------\n")

# 3. Check Output Node Name and Shape
print("### Output Node Information ###")

# The session.get_outputs() method returns a list of NodeArg objects.

output_node = session.get_outputs()[0]
print(f"Name: {output_node.name}")
print(f"Shape: {output_node.shape}")
print(f"Data Type: {output_node.type}")

### Input Node Information ###
Name: input
Shape: ['s77', 3, 200, 200]
Data Type: tensor(float)

-------------------------------

### Output Node Information ###
Name: output
Shape: ['s77', 1]
Data Type: tensor(float)


## Question 2: Target size

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

* 64x64
* 128x128
* 200x200
* 256x256

**Answer: 200x200**

In [3]:
!wget "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

--2025-12-09 14:38:04--  https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg
Resolving habrastorage.org (habrastorage.org)... 2a14:b680:0:56::34, 2a14:b680:0:56::35, 95.47.173.35, ...
Connecting to habrastorage.org (habrastorage.org)|2a14:b680:0:56::34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398272 (389K) [image/jpeg]
Saving to: 'yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

     0K .......... .......... .......... .......... .......... 12%  246K 1s
    50K .......... .......... .......... .......... .......... 25%  263K 1s
   100K .......... .......... .......... .......... .......... 38% 1,85M 1s
   150K .......... .......... .......... .......... .......... 51%  294K 1s
   200K .......... .......... .......... .......... .......... 64% 13,7M 0s
   250K .......... .......... .......... .......... .......... 77% 13,8M 0s
   300K .......... .......... .......... .......... .......... 89%  309K 0s
   350K .......... .......... .......... ......

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* -10.73
* -1.073
* 1.073
* 10.73

**Answer: -1.073**

In [14]:
import numpy as np
from PIL import Image
from io import BytesIO
from urllib import request
import torch
from torchvision import transforms

# Define constants and the transformation pipeline
IMAGE_URL = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
TARGET_SIZE = (200, 200)

train_transforms = transforms.Compose([
    transforms.Resize(TARGET_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

original_img = download_image(IMAGE_URL)
resized_img = prepare_image(original_img, TARGET_SIZE)

# 2. Apply the full transformation pipeline
# The output is a PyTorch Tensor with shape (C, H, W) -> (3, 200, 200)
final_tensor = train_transforms(resized_img)

# 3. Extract the R-channel value for the first pixel (index [0] at height 0, width 0)
# Tensor indexing: [Channel, Height, Width]
r_channel_first_pixel_value = final_tensor[0, 0, 0].item()

print(f"Value in the first pixel (R channel) using the transforms pipeline: {r_channel_first_pixel_value:.4f}")

Value in the first pixel (R channel) using the transforms pipeline: -1.0733


## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.09
* 0.49
* 0.69
* 0.89

**Answer: 0.49**

In [19]:
import numpy as np
import onnxruntime
from PIL import Image
from io import BytesIO
from urllib import request
from torchvision import transforms

# --- Model and Image Definitions ---
MODEL_PATH = 'hair_classifier_v1.onnx'
IMAGE_URL = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
TARGET_SIZE = (200, 200)
INPUT_NAME = 'input' # From inspecting the model Q1

preprocess = transforms.Compose([
    transforms.Resize(TARGET_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ---------------------------------------------

def process_and_prepare_for_onnx(image_url):
    """Downloads, preprocesses, and prepares the image as a NumPy array."""
    # 1. Download and load
    original_img = download_image(image_url)
    resized_img = prepare_image(original_img, TARGET_SIZE)
      
    # 2. Apply preprocessing (Resize, ToTensor, Normalize)
    tensor = preprocess(resized_img)
    
    # 3. Convert to NumPy and add batch dimension (Batch, C, H, W)
    # ONNX models require a batch dimension, hence .unsqueeze(0).
    input_array = tensor.unsqueeze(0).numpy() 
    
    return input_array


def get_model_output(input_array):
    """Loads the ONNX model and runs the inference."""
    # 1. Load the ONNX session
    session = onnxruntime.InferenceSession(MODEL_PATH)
    
    # 2. Run the model inference
    # input_feed maps the model's expected input name (INPUT_NAME) to our data.
    raw_outputs = session.run(
        output_names=None,  # Request all outputs
        input_feed={INPUT_NAME: input_array}
    )
    
    # The output is a list of results (one per output node)
    return raw_outputs[0]

# --- Execution ---

# 1. Prepare the image data
prepared_input = process_and_prepare_for_onnx(IMAGE_URL)

# 2. Get the model's raw output (logits)
raw_logit_output = get_model_output(prepared_input)

# 3. Process the output for final interpretation (Sigmoid)
# The output is usually a 2D array, so we extract the single logit value.
logit = raw_logit_output.flatten()[0]

# Apply the sigmoid function, as training code uses BCEWithLogitsLoss
# Sigmoid maps the logit to a probability between 0 and 1.
probability = 1 / (1 + np.exp(-logit))

print("\n--- Model Output ---")
print(f"Raw Logit (Model Score): {logit:.4f}")
print(f"Prediction Probability: {probability:.4f}")

# Your training code used a threshold of 0.5:
if probability > 0.5:
    print("Prediction: Positive Class (Likely has hair)")
else:
    print("Prediction: Negative Class (Likely no hair)")


--- Model Output ---
Raw Logit (Model Score): 0.0893
Prediction Probability: 0.5223
Prediction: Positive Class (Likely has hair)


## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
```

Note that it uses Python 3.13.

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.

## Question 5

Download the base image `agrigorev/model-2025-hairstyle:v1`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 88 Mb
* 208 Mb
* 608 Mb
* 1208 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

**Answer: 608 Mb**

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `hair_classifier_empty.onnx` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* -1.0
* -0.10
* 0.10
* 1.0

**Answer: 0.10**

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.

## Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2025-hairstyle -f homework.dockerfile .
docker tag model-2025-hairstyle:latest agrigorev/model-2025-hairstyle:v1
docker push agrigorev/model-2025-hairstyle:v1
```

(You don't need to execute this code)

In [20]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_empty.onnx

--2025-12-09 16:16:23--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_empty.onnx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/68add52d-cada-4976-9be6-69c40830aef8?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-09T09%3A52%3A51Z&rscd=attachment%3B+filename%3Dhair_classifier_empty.onnx&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-09T08%3A51%3A54Z&ske=2025-12-09T09%3A52%3A51Z&sks=b&skv=2018-11-09&sig=nitu96100J%2FxMnv0MMjuar521zI8au3ZNiddv4CwPRc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTI3MjA4NCwibmJmIjoxNzY1MjcxNzg0LCJwY

In [21]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_empty.onnx.data

--2025-12-09 16:16:33--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_empty.onnx.data
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/46212274-3836-4c28-88f6-52b2c3569b36?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-09T10%3A08%3A41Z&rscd=attachment%3B+filename%3Dhair_classifier_empty.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-09T09%3A08%3A17Z&ske=2025-12-09T10%3A08%3A41Z&sks=b&skv=2018-11-09&sig=z6hYPpAz%2Bsz56mDOETdcRwuOL4ud%2FeEvMR88zECcos0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTI3MzU5MywibmJmIjoxNzY1M